In [37]:
import subprocess
import os
import pandas as pd

In [30]:
def list_files(directory):
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            paths.append(os.path.join(root, file))
    return paths

def read_file(file_path):

    file_lines = []

    with open(file_path, "r") as file:
        for line in file:
            file_lines.append(line.strip())

    return file_lines

def save_file(file_path, object):
    with open(file_path, "w") as file:
        for line in object:
            file.write(line + "\n")

In [35]:
request_template = f"""
source /home/aleksandr/miniconda3/etc/profile.d/conda.sh && conda activate tf-py38;
conda env list;
conda list;
echo Activated environment: $CONDA_DEFAULT_ENV;
python -c "import tensorflow; print('Numpy version:', tensorflow.__version__)"
"""

In [32]:
ID = "SRR5961796"
folder = "GSE102556_pref"
abs_path = os.path.abspath("")
os.makedirs(folder, exist_ok=True)

In [7]:
request_template = f"""
        source /home/aleksandr/miniconda3/etc/profile.d/conda.sh && conda activate tf-py38;
        source /etc/profile.d/sra-tools.sh;
        cd {abs_path};
        prefetch {ID} --output-directory {folder};
        cd {folder};
        fasterq-dump {ID} --threads 9 --force
"""


In [8]:
request_template

'\n        source /home/aleksandr/miniconda3/etc/profile.d/conda.sh && conda activate tf-py38;\n        source /etc/profile.d/sra-tools.sh;\n        cd /home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis;\n        prefetch SRR5961796 --output-directory GSE102556_pref;\n        cd GSE102556_pref;\n        fasterq-dump SRR5961796 --threads 9 --force\n'

In [9]:


subprocess.Popen(request_template, shell=True, executable='/bin/bash')

/bin/bash: /home/aleksandr/miniconda3/envs/tf-py38/lib/libtinfo.so.6: no version information available (required by /bin/bash)


2024-08-01T14:13:43 prefetch.3.1.1: 1) Resolving 'SRR5961796'...
2024-08-01T14:13:44 prefetch.3.1.1: Current preference is set to retrieve SRA Normalized Format files with full base quality scores
2024-08-01T14:13:45 prefetch.3.1.1: 1) Downloading 'SRR5961796'...
2024-08-01T14:13:45 prefetch.3.1.1:  SRA Normalized Format file is being retrieved
2024-08-01T14:13:45 prefetch.3.1.1:  Downloading via HTTPS...


In [31]:
OUTPUT_dir = "GSE102556_OUTPUT"
os.makedirs(OUTPUT_dir, exist_ok=True)

In [20]:
STAR_dir = "GSE102556_STAR"
fastq_folder = "GSE102556_fastq"
os.makedirs(STAR_dir, exist_ok=True)

abs_path = os.path.abspath("")

files_fastq = list_files(fastq_folder)
files_fastq = [x for x in files_fastq if ".fastq" in x]
files_fastq.sort()


STAR_request = f"""
source /home/aleksandr/miniconda3/etc/profile.d/conda.sh && conda activate tf-py38 ;
cd {abs_path} ;
STAR --runMode alignReads --runThreadN 9 --genomeDir genome_index_STAR_hg19/ --outSAMtype BAM SortedByCoordinate \
--readFilesIn {files_fastq[0]} {files_fastq[1]} \
--outFileNamePrefix {STAR_dir}/ --quantMode GeneCounts --sjdbGTFfile genome_files_hg19/Homo_sapiens.GRCh37.87.gtf ;

"""
print(STAR_request)
process = subprocess.Popen(STAR_request, shell=True, executable="/bin/bash")
process.wait()
if process.stdout:
    process.stdout.close()
if process.stderr:
    process.stderr.close()
if process.stdin:
    process.stdin.close()


source /home/aleksandr/miniconda3/etc/profile.d/conda.sh && conda activate tf-py38 ;
cd /home/aleksandr/Desktop/WORK/OLINK_suicide_PSY_project/Data_preprocessing_analysis ;
STAR --runMode alignReads --runThreadN 9 --genomeDir genome_index_STAR_hg19/ --outSAMtype BAM SortedByCoordinate --readFilesIn GSE102556_fastq/SRR5961796_1.fastq GSE102556_fastq/SRR5961796_2.fastq --outFileNamePrefix GSE102556_STAR/ --quantMode GeneCounts --sjdbGTFfile genome_files_hg19/Homo_sapiens.GRCh37.87.gtf ;




/bin/bash: /home/aleksandr/miniconda3/envs/tf-py38/lib/libtinfo.so.6: no version information available (required by /bin/bash)


Aug 01 16:44:52 ..... started STAR run
Aug 01 16:44:52 ..... loading genome
Aug 01 16:45:06 ..... processing annotations GTF
Aug 01 16:45:15 ..... inserting junctions into the genome indices
Aug 01 16:46:17 ..... started mapping
Aug 01 16:49:57 ..... started sorting BAM
Aug 01 16:50:59 ..... finished successfully


In [25]:
STAR_files = list_files(STAR_dir)
STAR_counts = [x for x in STAR_files if "ReadsPerGene.out.tab" in x]
STAR_logs = [x for x in STAR_files if "Log.final.out" in x]

In [28]:
logs_list = read_file(STAR_logs[0])

In [45]:
counts_df = pd.read_csv(STAR_counts[0], sep="\t", header=None)
counts_df.columns = ["ID", "Unstranded", "Strand_1", "Strand_2"]

In [51]:
counts_df.tail(60)

,ID,Unstranded,Strand_1,Strand_2
57849,ENSG00000272060,911,228,683
57850,ENSG00000242716,0,0,0
57851,ENSG00000266658,5589,2687,2902
57852,ENSG00000243151,0,0,0
57853,ENSG00000265807,0,0,0
57854,ENSG00000265525,0,0,0
57855,ENSG00000241335,0,0,0
57856,ENSG00000265830,0,0,0
57857,ENSG00000215615,34,34,0
57858,ENSG00000215764,0,0,0


In [36]:
log_path = os.path.join(OUTPUT_dir, ID + "_star_logs.txt")
counts_path = os.path.join(OUTPUT_dir, ID + "_star_counts.csv")